In [1]:
from glob import glob                                                           
from PIL import Image
from pathlib import Path
import os 
import cv2
import numpy as np 
import random
import re

import shutil
import pandas as pd
import errno

from sklearn.model_selection import train_test_split
from numpy import array

# Data Visualization
import cufflinks as cf
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg

# Configuración del notebook
cf.go_offline()

## Data Wrangling

### Manejando imagenes

In [2]:
imag_dir = './data/images/train'

In [3]:
#obteniendo etiquetas
df = pd.read_csv('./data/metadata.csv')

In [4]:
#borrando los registros con precio 0 (validation)
#df.drop(df[df['price']==0].index, inplace=True)

### Creando Dataframe con etiquetas

In [5]:
labbel_df = pd.DataFrame(os.listdir('./data/images/train'),columns=['file'])

In [6]:
labbel_df['id'] = [name.split('_')[0] for name in labbel_df['file'].values]

In [7]:
labbel_df['price'] = [df.loc[[int(id)-1]]['price'].values[0] for id in labbel_df['id'].values]

In [8]:
labbel_df.head()

,file,id,price
0,102_bathroom.jpg,102,769000
1,428_kitchen.jpg,428,559000
2,498_kitchen.jpg,498,585000
3,162_kitchen.jpg,162,164000
4,531_bathroom.jpg,531,399900


### Separando test del train

In [9]:
#escogiendo test (20%)
labbels = np.unique(labbel_df['id'])
no_registers = len(labbels)
random.seed(0)
test_id = random.sample(list(labbels),no_registers//5)

In [10]:
test_labbels = labbel_df[labbel_df['id'].isin(test_id)].sort_values('id')
train_labbels = labbel_df[~labbel_df['id'].isin(test_id)].sort_values('id')

In [11]:
train_labbels

,file,id,price
573,10_kitchen.jpg,10,1500000
171,10_frontal.jpg,10,1500000
1908,10_bedroom.jpg,10,1500000
354,10_bathroom.jpg,10,1500000
1810,100_frontal.jpg,100,679000
...,...,...,...
288,96_frontal.jpg,96,521500
1052,99_bathroom.jpg,99,629000
1742,99_kitchen.jpg,99,629000
672,99_frontal.jpg,99,629000


#### moviendo imagenes de test a train

In [12]:
def move_image(imag,old,new):
    try:
        shutil.copy(os.path.join(old+"/"+imag), os.path.join(new+"/"+imag))
    except OSError as e:
        if e.errno != errno.ENOENT:
            raise

In [13]:
#Creando carpetas de train y tests
if not os.path.exists('./data/images/test'):
    os.mkdir('./data/images/test')

In [14]:
for file in test_labbels['file'].values:
    new = './data/images/test'
    move_image(file,imag_dir,new)

#### moviendo a cada tipo de imagen

In [15]:
image_type = ['bathroom','bedroom','frontal','kitchen']

In [16]:
#creando folders para train
for type in image_type:
    if not os.path.exists(f'./data/images/train/{type}'):
        os.mkdir(f'./data/images/train/{type}')

In [20]:
#para train
for type in image_type:
    for file in train_labbels['file'].values:
        if type in file:
            new = f'./data/images/train/{type}'
            move_image(file,imag_dir,new)

In [21]:
#creando folders para test
for type in image_type:
    if not os.path.exists(f'./data/images/test/{type}'):
        os.mkdir(f'./data/images/test/{type}')

In [22]:
#para test
for type in image_type:
    for file in train_labbels['file'].values:
        if type in file:
            new = f'./data/images/test/{type}'
            move_image(file,'./data/images/test/',new)